# Parsing Westmoreland County Election results

The following notebook parses the archived 2018 precinct level election results from Westmoreland County from plain text into a dataframe (spreadsheet like data structure) to be used for matching.

Steps:

1. Copy the plain text data from Westmoreland County's [website]( https://www.co.westmoreland.pa.us/DocumentCenter/View/15313/2018-General-PrecinctSummary)
2. Store the plain text locally at `westmoreland_raw.txt`
3. Use Excel's column parsing feature to create `westmoreland_county.csv`
4. Use this script to parse the primitive csv into a csv matching Open Elections style for its statewide precinct level resutls. 
5. Store the resulting file locally at `westmoreland_county_parsed.csv`

In [1]:
import pandas as pd
import numpy as np
import math
import os
import re
os.getcwd()

'/Users/baxterdemers/pa-2018/parsing_election_results/westmoreland'

In [2]:
df = pd.read_csv('westmoreland_county.csv')
df.head()

,candidate,votes,election_day,absentee
0,0100 ADAMSBURG BOROUGH,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,REGISTERED VOTERS - TOTAL . . . . . .,112,NaN,NaN
3,BALLOTS CAST - TOTAL. . . . . . . .,61,61,0
4,VOTER TURNOUT - TOTAL . . . . . . .,NaN,NaN,NaN


### Main parsing script

In [3]:
d = {
    'STRAIGHT PARTY':'Straight Party', 
    'UNITED STATES SENATOR':'U.S. Senate',
    'GOVERNOR':'Governor',
    'REPRESENTATIVE IN CONG':'U.S. House', 
    'REPRESENTATIVE IN THE':'State House',
}
output = pd.DataFrame(columns=['county', 'precinct', 'office', 'district', 'candidate', 'party','votes', 'absentee', 'election_day'])
prev_blank = False
lst = []
first = True
for idx, row in df.iterrows():
    can = row.candidate
    if type(can) != str and math.isnan(can):
        prev_blank = True
        continue
    elif str(can)[:4].isnumeric():
        prec = can
    elif can.split()[0] in {'REGISTERED', 'BALLOTS', 'VOTER','TOTAL', 'Total', 'DISTRICT', 'Vote', 'WRITE-IN.', 'PREC', 'Run'}:
        continue
    elif prev_blank:
        if "CONGRESS" in can:
            office = 'U.S. House'
            temp = re.findall(r'\d+', can.split('CONGRESS')[1]) 
            district = list(map(int, temp))[0]
        elif "GENERAL ASSEMBLY" in can:
            office = 'State House'
            temp = re.findall(r'\d+', can.split('GENERAL ASSEMBLY')[1]) 
            district = list(map(int, temp))[0]
        else:
            district = np.nan
            office = d[can.strip()]
        prev_blank = False
    else:
        splits = can.split('(')
        can_name = splits[0].strip()
        party = splits[1].split(')')[0]
        if can_name == 'EBERT G BILL BEEMAN':
            party = 'LIB'
        res = {
            'county':'Westmoreland',
            'precinct':prec,
            'office':office,
            'district':district,
            'candidate':can_name,
            'party':party,
            'votes':row.votes, 
            'absentee':row.absentee, 
            'election_day':row.election_day,
        }
        lst.append(res)
        if first:
            print(res)
            first = False
            
output = output.append(lst)

{'county': 'Westmoreland', 'precinct': '0100 ADAMSBURG BOROUGH', 'office': 'Straight Party', 'district': nan, 'candidate': 'Democratic', 'party': 'DEM', 'votes': '12', 'absentee': '0', 'election_day': '12'}


### Validation

In [4]:
output[output.party==''].candidate.unique()

array([], dtype=object)

In [5]:
output.party.unique()

array(['DEM', 'REP', 'GRN', 'LN'], dtype=object)

In [6]:
output.head(20)

,county,precinct,office,district,candidate,party,votes,absentee,election_day
0,Westmoreland,0100 ADAMSBURG BOROUGH,Straight Party,NaN,Democratic,DEM,12,0,12
1,Westmoreland,0100 ADAMSBURG BOROUGH,Straight Party,NaN,Republican,REP,20,0,20
2,Westmoreland,0100 ADAMSBURG BOROUGH,Straight Party,NaN,Green Party,GRN,0,0,0
3,Westmoreland,0100 ADAMSBURG BOROUGH,Straight Party,NaN,Libertarian,LN,1,0,1
4,Westmoreland,0100 ADAMSBURG BOROUGH,U.S. Senate,NaN,Bob Casey Jr,DEM,24,0,24
5,Westmoreland,0100 ADAMSBURG BOROUGH,U.S. Senate,NaN,Lou Barletta,REP,35,0,35
6,Westmoreland,0100 ADAMSBURG BOROUGH,U.S. Senate,NaN,Neal Gale,GRN,0,0,0
7,Westmoreland,0100 ADAMSBURG BOROUGH,U.S. Senate,NaN,Dale R Kerns Jr,LN,2,0,2
8,Westmoreland,0100 ADAMSBURG BOROUGH,Governor,NaN,Tom Wolf,DEM,28,0,28
9,Westmoreland,0100 ADAMSBURG BOROUGH,Governor,NaN,Scott R Wagner,REP,32,0,32


In [7]:
output.to_csv('westmoreland_county_parsed.csv',index=False)